#### What is pickle?
* Pickle is a python module which converts a Python object into a byte stream, aka "pickled",and a byte stream back into a Python object, aka "unpickled".


#### 1. How does pickle serialize class instance?

* By default, pickle will retrieve the class and the attributes of an instance via introspection. The following code shows an implementation of this behaviour.

In [2]:
def save(obj):
    return (obj.__class__, obj.__dict__)

* Only the attributes of instance are pickled. The class's code and data are not pickled along with the instance. For example, **class attributes are not pickled**.

    For example:

In [38]:
# Define class Foo1
class Foo1:
    pass

# Define class Foo2 with attribute attr
class Foo2:
    attr = 100

foo1 = Foo1()
foo2 = Foo2()

# Pickle foo and foo2
import pickle
pickled_foo1 = pickle.dumps(foo1)
pickled_foo2 = pickle.dumps(foo2)

# Compare the pickled results
print(pickled_foo1)
print(pickled_foo2)

b'\x80\x03c__main__\nFoo1\nq\x00)\x81q\x01.'
b'\x80\x03c__main__\nFoo2\nq\x00)\x81q\x01.'


   The only difference is the name of the class (One is "Foo1"  and the other is "Foo2"). This proves that the class attributes are not pickled. 

#### 2. How does pickle de-serialize class instance?
* Pickle will first get the name of the class, creates an uninitialized instance and then restores the saved attributs. **The \_\_init\_\_ method by default is not invoked.** The following code shows an implementation of this behaviour.

In [12]:
def load(cls, attributes):
    obj = cls.__new__(cls)
    obj.__dict__.update(attributes)
    return obj

To prove this process, we first define a class as follows:

In [29]:
# Define class Foo3
class Foo3:
    def __new__(cls):
        print("Calling method __new__")
        return super().__new__(cls)
        
    def __init__(self):
        print("Calling method __init__")

foo3 = Foo3()

Calling method __new__
Calling method __init__


In [30]:
# Pickle foo3
pickled_foo3 = pickle.dumps(foo3)

# Unpickle
unpickled_foo3 = pickle.loads(pickled_foo3)

Calling method __new__


Aha, only the \_\_new\_\_ method is called!

* What if the \_\_new\_\_ method have arguments? The answer is to define \_\_getnewargs\_\_ method.

In [34]:
# Define clas Foo4
class Foo4:
    def __new__(cls, a, b):
        print("Calling method __new__")
        obj = super().__new__(cls)
        obj.a = a
        obj.b = b
        return obj
    
    def __init__(self, *args, **kwargs):
        print("Calling method __init__")
    
    def __getnewargs__(self):
        return self.a, self.b

foo4 = Foo4('a', 'b')

Calling method __new__
Calling method __init__


In [37]:
# Pickle foo3
pickled_foo4 = pickle.dumps(foo4)

# Unpickle
unpickled_foo4 = pickle.loads(pickled_foo4)
print(unpickled_foo4.a, unpickled_foo4.b)

Calling method __new__
a b
